In [2]:

## Importing important Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import ast
from datetime import datetime, timedelta

In [4]:

file_path = "assignment_data.csv"

# Loading the Excel file
xls = pd.read_csv(file_path)

In [5]:
xls.head(8)

,provider_id,detail,source,event_time
0,6873f0af240706d65514ec247c64f330,{u'mme': False},job_responded,2017-09-01 00:00:00.000000
1,a5a021d997269c4f1846903ff0f5d737,"{u'lat': 18.9810108, u'long': 72.8187476}",location_tracking,2017-09-01 00:00:02.000000
2,77af6d6d6db530c389ec1014d557a01e,{u'mme': False},decline_request,2017-09-01 00:00:05.000000
3,7ea218c2b9a6dada27e0d5bdf5f896de,{u'mme': False},decline_request,2017-09-01 00:00:12.000000
4,0d7a9d9c00231885b413175980a9beca,"{u'lat': 12.9959477, u'long': 77.6527175}",location_tracking,2017-09-01 00:00:15.000000
5,b72c9eeb4c52a2bdb7c4a8fe09871ec8,"{u'lat': 19.0258816, u'long': 72.8727786}",location_tracking,2017-09-01 00:00:20.000000
6,fd385a93b517e41f5ae906be5213f0b0,"{u'lat': 18.5398284, u'long': 73.7821132}",location_tracking,2017-09-01 00:00:23.000000
7,f1be9a4ebf57391f046464144a20fd36,{u'mme': False},job_responded,2017-09-01 00:00:27.000000


In [6]:
xls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1945726 entries, 0 to 1945725
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   provider_id  object
 1   detail       object
 2   source       object
 3   event_time   object
dtypes: object(4)
memory usage: 59.4+ MB


In [7]:
xls.shape

(1945726, 4)

In [8]:
xls.isnull().mean()

provider_id    0.000000
detail         0.038612
source         0.000000
event_time     0.000000
dtype: float64

In [9]:

# Converting event_time to datetime
xls['event_time'] = pd.to_datetime(xls['event_time'])

In [10]:
xls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1945726 entries, 0 to 1945725
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   provider_id  object        
 1   detail       object        
 2   source       object        
 3   event_time   datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 59.4+ MB


In [11]:
# Filtering out rows after 7 PM
xls = xls[xls['event_time'].dt.hour < 19]

In [12]:
xls.shape

(1613934, 4)

## Parsed the 'detail' column so we can:

### Extract values like 'mme'

### Identifing event types correctly

### Filter and classify status without string hacks

In [13]:

def parse_detail(val):
    try:
        return ast.literal_eval(val)
    except:
        return {}

xls['detail_parsed'] = xls['detail'].apply(parse_detail)

In [14]:
xls.head()

,provider_id,detail,source,event_time,detail_parsed
0,6873f0af240706d65514ec247c64f330,{u'mme': False},job_responded,2017-09-01 00:00:00,{'mme': False}
1,a5a021d997269c4f1846903ff0f5d737,"{u'lat': 18.9810108, u'long': 72.8187476}",location_tracking,2017-09-01 00:00:02,"{'lat': 18.9810108, 'long': 72.8187476}"
2,77af6d6d6db530c389ec1014d557a01e,{u'mme': False},decline_request,2017-09-01 00:00:05,{'mme': False}
3,7ea218c2b9a6dada27e0d5bdf5f896de,{u'mme': False},decline_request,2017-09-01 00:00:12,{'mme': False}
4,0d7a9d9c00231885b413175980a9beca,"{u'lat': 12.9959477, u'long': 77.6527175}",location_tracking,2017-09-01 00:00:15,"{'lat': 12.9959477, 'long': 77.6527175}"


In [15]:
xls = xls[
    xls['detail'].str.contains('True|False', na=False) |
    xls['source'].str.contains('Action on Job', na=False)
]

In [16]:
xls.shape

(695667, 5)

# Determining online/offline status

In [17]:
# Determine online/offline status
def get_status(row):
    if 'mme' in row['detail_parsed']:
        return 'online' if row['detail_parsed']['mme'] else 'offline'
    elif 'Action on Job' in str(row['source']):
        return 'online'
    elif 'False' in str(row['detail']):
        return 'offline'
    elif 'True' in str(row['detail']):
        return 'online'
    else:
        return 'ignore'

xls['status'] = xls.apply(get_status, axis=1)

In [18]:
xls.head(8)

,provider_id,detail,source,event_time,detail_parsed,status
0,6873f0af240706d65514ec247c64f330,{u'mme': False},job_responded,2017-09-01 00:00:00,{'mme': False},offline
2,77af6d6d6db530c389ec1014d557a01e,{u'mme': False},decline_request,2017-09-01 00:00:05,{'mme': False},offline
3,7ea218c2b9a6dada27e0d5bdf5f896de,{u'mme': False},decline_request,2017-09-01 00:00:12,{'mme': False},offline
7,f1be9a4ebf57391f046464144a20fd36,{u'mme': False},job_responded,2017-09-01 00:00:27,{'mme': False},offline
10,44f11b351f554bb6f564c9cd7c861645,{u'mme': False},job_responded,2017-09-01 00:00:48,{'mme': False},offline
11,18718c56ffd665f10a41336e71c94ff7,"{u'mme': False, u'requestID': u'59a82b992db8d2...",other-provider-accepted-wave-zero,2017-09-01 00:00:48,"{'mme': False, 'requestID': '59a82b992db8d25d0...",offline
12,d43fbfbb539c2998bc2229ae35ae6375,"{u'mme': False, u'requestID': u'59a82b992db8d2...",other-provider-accepted-wave-zero,2017-09-01 00:00:48,"{'mme': False, 'requestID': '59a82b992db8d25d0...",offline
13,4da38b70324ea165ea75ee33ac204211,"{u'mme': False, u'requestID': u'59a82b992db8d2...",other-provider-accepted-wave-zero,2017-09-01 00:00:48,"{'mme': False, 'requestID': '59a82b992db8d25d0...",offline


In [19]:
xls['status'].value_counts()

status
offline    518831
online     176836
Name: count, dtype: int64

In [20]:
# Sorting for time difference calculation
xls = xls.sort_values(by=['provider_id', 'event_time'])

In [21]:

# Creating shifted columns
xls['next_event_time'] = xls.groupby('provider_id')['event_time'].shift(-1)
xls['next_status'] = xls.groupby('provider_id')['status'].shift(-1)

## Calculating duration only for 'online' rows

In [22]:

xls['seconds_online'] = np.where(
    (xls['status'] == 'online') & (xls['next_status'].notna()),
    (xls['next_event_time'] - xls['event_time']).dt.total_seconds(),
    0
)

## Floor to hour to create hour slots

In [23]:

xls['hour_slot'] = xls['event_time'].dt.floor('H')

C:\Users\Gaurav\AppData\Local\Temp\ipykernel_23224\4217088613.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  xls['hour_slot'] = xls['event_time'].dt.floor('H')


In [24]:

final_xls = xls.groupby(['provider_id', 'hour_slot'], as_index=False)['seconds_online'].sum()

## Clipping at 3600 max seconds per hour

In [25]:


final_xls['seconds_online'] = final_xls['seconds_online'].clip(upper=3600)

In [26]:
# Extracting date, hour, and end_hour from hour_slot
final_xls['date'] = final_xls['hour_slot'].dt.strftime('%d-%b') 
final_xls['Hour startime'] = final_xls['hour_slot'].dt.hour
final_xls['Hour end time'] = final_xls['Hour startime'] + 1

In [27]:

# Reordering and renaming columns
final_xls = final_xls[['provider_id', 'date', 'Hour startime', 'Hour end time', 'seconds_online']].copy()
final_xls.rename(columns={'seconds_online': 'Seconds online'}, inplace=True)


final_xls = final_xls.sort_values(by=['provider_id', 'date', 'Hour startime'])

# Final Output preview

In [28]:

final_xls

,provider_id,date,Hour startime,Hour end time,Seconds online
0,00020da44b0a44e867de5f26c69886ac,07-Sep,0,1,0.0
1,0002beae790876e71dfb99a904563c30,01-Sep,17,18,0.0
2,0002beae790876e71dfb99a904563c30,02-Sep,16,17,0.0
3,0002beae790876e71dfb99a904563c30,03-Sep,11,12,0.0
4,0002beae790876e71dfb99a904563c30,03-Sep,14,15,0.0
...,...,...,...,...,...
316218,fff7363be392a544d1cc3d15165a7865,23-Sep,12,13,0.0
316219,fff7363be392a544d1cc3d15165a7865,28-Sep,9,10,0.0
316220,fff7363be392a544d1cc3d15165a7865,28-Sep,11,12,0.0
316221,fff7363be392a544d1cc3d15165a7865,29-Sep,9,10,0.0


In [29]:
# Converting to Excel File
final_xls.to_excel("final_output.xlsx", index=False)

# No of Online Events

In [30]:
online_events = xls[xls['status'] == 'online']
online_count = len(online_events)
print(" No of Online Events:", online_count)

 No of Online Events: 176836


# No of Offline Events

In [31]:
offline_events = xls[xls['status'] == 'offline']
offline_count = len(offline_events)
print("No of Offline Events:", offline_count)

No of Offline Events: 518831


In [33]:
valid_rows = xls[
    xls['detail'].str.contains('True|False', na=False) |
    xls['source'].str.contains('Action on Job', na=False)
]



# Count unique providers after removing error rows

In [ ]:

providers_after_error_filter = valid_rows['provider_id'].nunique()
print("Providers after removing error rows:", providers_after_error_filter)

Providers after removing error rows: 14914


# Filter the Rows for event time before 7 PM

In [35]:

valid_rows_time_filtered = valid_rows[valid_rows['event_time'].dt.hour < 19]


providers_after_time_filter = valid_rows_time_filtered['provider_id'].nunique()
print("Providers after removing events on/after 7 PM:", providers_after_time_filter)

Providers after removing events on/after 7 PM: 14914


# No of Rows in final output data frame (No of Providers * 30 * 11)

In [38]:
valid_providers = valid_rows_time_filtered['provider_id'].nunique()


final_expected_rows = valid_providers * 30 * 11
print(" Expected Rows in Final Output (Providers × 30 × 11):", final_expected_rows)

 Expected Rows in Final Output (Providers × 30 × 11): 4921620


# Rows in Final Output with Seconds_online > 0

In [39]:
non_zero_seconds = final_xls[final_xls['Seconds online'] > 0]
non_zero_count = len(non_zero_seconds)
print(" Rows in Final Output with Seconds_online > 0:", non_zero_count)

 Rows in Final Output with Seconds_online > 0: 73531
